# T2-기출 3회 [분류] 여행 보험 패키지 상품을 구매할 확률 값을 구하시오 / 리더보드 제출용
--- 
- mission  
코드 탭에서 "new notebook"으로 문제 풀기 (생성된 csv파일 다운로드)
예측한 csv파일 "Submit Predictions"하기
예측할 값(y): TravelInsurance (여행보험 패지지를 구매 했는지 여부 0:구매안함, 1:구매)

- 평가: roc-auc 평가지표
- data: t2-1-train.csv, t2-1-test.csv
- 제출 형식
- id,TravelInsurance  
0,0.3  
1,0.48  
2,0.3  
3,0.83  

In [144]:
import pandas as pd
import numpy as np

In [145]:
train = pd.read_csv('../kaggle/TravelInsurance(t2-3회)/t2-1-train.csv')
test = pd.read_csv('../kaggle/TravelInsurance(t2-3회)/t2-1-test.csv')

In [146]:
# 제출용 id,TravelInsurance  
id = test['id']
train = train.drop('id',axis=1)
test = test.drop('id',axis=1)
train

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,28,Private Sector/Self Employed,Yes,1250000.0,6,1,No,No,0
1,31,Private Sector/Self Employed,Yes,1250000.0,7,1,No,No,0
2,29,Private Sector/Self Employed,Yes,1200000.0,7,0,No,No,1
3,33,Government Sector,Yes,650000.0,6,1,No,No,1
4,28,Private Sector/Self Employed,Yes,800000.0,6,0,No,Yes,1
...,...,...,...,...,...,...,...,...,...
1485,28,Private Sector/Self Employed,Yes,800000.0,4,0,No,No,0
1486,34,Private Sector/Self Employed,Yes,1000000.0,9,0,No,No,0
1487,26,Private Sector/Self Employed,Yes,450000.0,5,1,No,No,1
1488,25,Private Sector/Self Employed,No,1150000.0,3,1,No,Yes,0


# EDA
---

In [147]:
train.isnull().sum()

Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           4
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
TravelInsurance        0
dtype: int64

In [148]:
test.isnull().sum()

Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           3
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64

In [149]:
train.shape,test.shape

((1490, 9), (497, 8))

# 전처리
---

In [150]:
target = train['TravelInsurance']
train = train.drop('TravelInsurance',axis=1)

In [151]:
train['AnnualIncome'] = train['AnnualIncome'].fillna(train['AnnualIncome'].mean())
test['AnnualIncome'] = test['AnnualIncome'].fillna(test['AnnualIncome'].mean())

In [152]:
train.isnull().sum()

Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64

In [153]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  1490 non-null   int64  
 1   Employment Type      1490 non-null   object 
 2   GraduateOrNot        1490 non-null   object 
 3   AnnualIncome         1490 non-null   float64
 4   FamilyMembers        1490 non-null   int64  
 5   ChronicDiseases      1490 non-null   int64  
 6   FrequentFlyer        1490 non-null   object 
 7   EverTravelledAbroad  1490 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 93.2+ KB


In [154]:
train_num = train.select_dtypes('int','float')
train_cat = train.select_dtypes('object')

test_num = test.select_dtypes('int','float')
test_cat = test.select_dtypes('object')

In [155]:
train_cat = pd.get_dummies(train_cat)
test_cat = pd.get_dummies(test_cat)

In [156]:
train = pd.concat([train_num,train_cat],axis=1)
test = pd.concat([test_num,test_cat],axis=1)
train

,Age,FamilyMembers,ChronicDiseases,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,GraduateOrNot_No,GraduateOrNot_Yes,FrequentFlyer_No,FrequentFlyer_Yes,EverTravelledAbroad_No,EverTravelledAbroad_Yes
0,28,6,1,0,1,0,1,1,0,1,0
1,31,7,1,0,1,0,1,1,0,1,0
2,29,7,0,0,1,0,1,1,0,1,0
3,33,6,1,1,0,0,1,1,0,1,0
4,28,6,0,0,1,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1485,28,4,0,0,1,0,1,1,0,1,0
1486,34,9,0,0,1,0,1,1,0,1,0
1487,26,5,1,0,1,0,1,1,0,1,0
1488,25,3,1,0,1,1,0,1,0,0,1


In [157]:
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 11 columns):
 #   Column                                        Non-Null Count  Dtype
---  ------                                        --------------  -----
 0   Age                                           1490 non-null   int64
 1   FamilyMembers                                 1490 non-null   int64
 2   ChronicDiseases                               1490 non-null   int64
 3   Employment Type_Government Sector             1490 non-null   uint8
 4   Employment Type_Private Sector/Self Employed  1490 non-null   uint8
 5   GraduateOrNot_No                              1490 non-null   uint8
 6   GraduateOrNot_Yes                             1490 non-null   uint8
 7   FrequentFlyer_No                              1490 non-null   uint8
 8   FrequentFlyer_Yes                             1490 non-null   uint8
 9   EverTravelledAbroad_No                        1490 non-null   uint8
 10  EverTravelle

(None, None)

In [158]:
test = test.drop(['Employment Type_Casual employment'],axis=1)

# 모델
---

In [159]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, target, stratify=target, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1117, 11), (373, 11), (1117,), (373,))

In [160]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state= 0,max_depth =6,n_estimators=300)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.8230563002680965

In [161]:
pred = model.predict(test)

In [162]:
train = pd.concat([X_train,X_test])
target = pd.concat([y_train,y_test])

In [163]:
# random_state= 0,max_depth =7,n_estimators=500

In [164]:
model_final = RandomForestClassifier(random_state= 0,max_depth =7,n_estimators=500)
model_final.fit(train,target)
pred_final = model_final.predict_proba(test)
print(pred_final)

pred_final = pred_final[:,1]

[[0.88869236 0.11130764]
 [0.88898621 0.11101379]
 [0.84293955 0.15706045]
 [0.07797128 0.92202872]
 [0.76117659 0.23882341]
 [0.03662508 0.96337492]
 [0.76342408 0.23657592]
 [0.78113471 0.21886529]
 [0.28361018 0.71638982]
 [0.9013471  0.0986529 ]
 [0.79691795 0.20308205]
 [0.88623739 0.11376261]
 [0.77065278 0.22934722]
 [0.80778718 0.19221282]
 [0.1603047  0.8396953 ]
 [0.83647763 0.16352237]
 [0.78085328 0.21914672]
 [0.80582579 0.19417421]
 [0.75618349 0.24381651]
 [0.76712479 0.23287521]
 [0.73707835 0.26292165]
 [0.83116946 0.16883054]
 [0.06577698 0.93422302]
 [0.22947829 0.77052171]
 [0.88623739 0.11376261]
 [0.0298506  0.9701494 ]
 [0.0660118  0.9339882 ]
 [0.85494627 0.14505373]
 [0.08697477 0.91302523]
 [0.09531096 0.90468904]
 [0.87953624 0.12046376]
 [0.15638736 0.84361264]
 [0.87953624 0.12046376]
 [0.76257726 0.23742274]
 [0.95422246 0.04577754]
 [0.32388975 0.67611025]
 [0.76769349 0.23230651]
 [0.49666001 0.50333999]
 [0.07797128 0.92202872]
 [0.81324099 0.18675901]


In [165]:
# 제출
sub = pd.DataFrame(data ={'id':id,'TravelInsurance':pred_final})
sub.to_csv('./TravelInsurance(t2-3회)/submission.csv',index=False)

In [166]:
sub

,id,TravelInsurance
0,0,0.111308
1,1,0.111014
2,2,0.157060
3,3,0.922029
4,4,0.238823
...,...,...
492,492,0.129876
493,493,0.946821
494,494,0.142097
495,495,0.925703
